<a href="https://colab.research.google.com/github/armandochernandez-ai/Curso-python-slava/blob/main/Clima/Clima_SPEI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#El script original fue elaborado para Rstudio, se adaptó para que funcione en esta plataforma

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

# Montar Google Drive
drive.mount('/content/drive')

# Configuración de rutas
input_path = '/content/drive/MyDrive/Clima_Jalisco'
output_path = '/content/drive/MyDrive/Clima_Jalisco/IMAGENES_SPEI_2'
output_data_path = '/content/drive/MyDrive/Clima_Jalisco/DATOS_SPEI_2'

# Crear directorios de salida si no existen
os.makedirs(output_path, exist_ok=True)
os.makedirs(output_data_path, exist_ok=True)

# Cargar datos
estaciones_jalmes_FL = pd.read_csv(f'{input_path}/estaciones_jalmes_FL.csv')

#En donde existan NA debido a que no existirá referencia alguna, es decir, obtendrá NA que más delnate se convierte en 0
estaciones_jalmes_FL['p'] = estaciones_jalmes_FL.apply(lambda row: np.nan if pd.isna(row['tprom']) else row['p'], axis=1)

# Función para calcular PET usando Thornthwaite
def thornthwaite(temperature, lat, months):
    """
    Calcula la Evapotranspiración Potencial usando el método de Thornthwaite
    """
    # Coeficientes para el cálculo
    pet_values = []
    for i, temp in enumerate(temperature):
        month = months[i]
        # Horas de luz del día (aproximación)
        if lat >= 0:  # Hemisferio norte
            day_hours = [10.5, 11.0, 12.0, 13.0, 14.0, 14.5,
                        14.5, 13.5, 12.5, 11.5, 10.5, 10.0]
        else:  # Hemisferio sur
            day_hours = [14.5, 13.5, 12.5, 11.5, 10.5, 10.0,
                        10.5, 11.0, 12.0, 13.0, 14.0, 14.5]

        if temp > 0:
            heat_index = (temp / 5) ** 1.514
            pet = 16 * (10 * temp / heat_index) ** (0.5 + 0.01 * lat) * day_hours[month-1] / 12
        else:
            pet = 0
        pet_values.append(pet)
    return pet_values

# Función para calcular SPEI
def calculate_spei(balance, scale=1):
    """
    Calcula el SPEI para una escala dada
    """
    # Rolling mean para la escala
    balance_roll = pd.Series(balance).rolling(window=scale, min_periods=1).mean()

    # Ajuste de distribución gamma (simplificado)
    spei_values = []
    for i in range(len(balance_roll)):
        if i >= scale-1:
            window_data = balance_roll[max(0, i-scale):i+1]  # Adjusted window size
            if len(window_data) >= scale:  # Minimum data for calculation
                # Estandarización simple (aproximación)
                z_score = (balance_roll.iloc[i] - window_data.mean()) / window_data.std()
                spei_values.append(z_score if not np.isnan(z_score) else 0)
            else:
                spei_values.append(0)
        else:
            spei_values.append(0)

    return spei_values

# Lista para almacenar los dataframes de cada estación
all_stations_data = []

# Función para procesar cada estación
def procesar_estacion(codigo):
    try:
        # Filtrar datos de la estación
        estacion = estaciones_jalmes_FL[estaciones_jalmes_FL['Codigo'] == codigo].copy()

        # Ordenar por año y mes
        estacion = estacion.sort_values(['Year', 'Month'])

        # Calcular PET y Balance Hídrico
        estacion['PET'] = thornthwaite(estacion['tprom'].values,
                                     estacion['latitud'].iloc[0],
                                     estacion['Month'].values)
        estacion['BAL'] = estacion['p'] - estacion['PET']

        # Sort by Year and Month after calculating BAL
        estacion = estacion.sort_values(['Year', 'Month']).reset_index(drop=True)


        # Calcular SPEI-12 (cambiado de 1 a 12)
        spei12 = calculate_spei(estacion['BAL'].values, scale=12)
        estacion['SPEI12'] = spei12

        # Guardar tabla completa de la estación
        estacion.to_csv(f'{output_data_path}/datos_spei12_{codigo}.csv', index=False)

        # Añadir datos de la estación a la lista global
        all_stations_data.append(estacion)


        # Gráfico SPEI básico (cambiado SPEI1 a SPEI12)
        plt.figure(figsize=(13, 9.6))
        plt.bar(estacion['Year'] + (estacion['Month']-1)/12, estacion['SPEI12'],
                width=0.03, alpha=0.8, color='blue')
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        plt.axhline(y=1, color='green', linestyle='--', alpha=0.5)
        plt.axhline(y=-1, color='red', linestyle='--', alpha=0.5)
        plt.title(f'Estación: {codigo}', fontsize=14, fontweight='bold')
        plt.xlabel('Año')
        plt.xticks(np.arange(1980, 2025, 1), rotation=45)
        plt.ylabel('SPEI12')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{output_path}/spei12_{codigo}.png', dpi=400, bbox_inches='tight')
        plt.close()

        # Preparar datos para gráfico de barras (cambiado SPEI1 a SPEI12)
        estacion['Period'] = estacion['Year'] + (estacion['Month'] - 1) / 12
        estacion['sing'] = estacion['SPEI12'].apply(lambda x: 'Húmedo' if x >= 0 else 'Seco')

        # Gráfico ggplot2 mejorado (estilo similar) (cambiado SPEI1 a SPEI12)
        plt.figure(figsize=(13, 9.6))

        # Crear gráfico de barras
        colors = {'Húmedo': 'blue', 'Seco': 'red'}
        for i, row in estacion.iterrows():
            plt.bar(row['Period'], row['SPEI12'],
                   color=colors[row['sing']],
                   width=0.03, alpha=0.8)

        plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        plt.ylim(-3, 3)
        plt.yticks(range(-3, 3))
        plt.xticks(np.arange(1980, 2024, 1), rotation=45)
        plt.xlim(1979.5, 2024.5)

        # Obtener nombre de la estación
        site_name = estacion['site'].iloc[0] if 'site' in estacion.columns else f"Estación {codigo}"

        plt.ylabel('SPEI', fontstyle='italic', fontweight='bold', fontsize=9)
        plt.xlabel('Año', fontstyle='italic', fontweight='bold', fontsize=10)
        plt.title(f'ESTACIÓN {site_name} (SMN: {codigo})\nÍndice Estandarizado de Evapotranspiración',
                 fontweight='bold', fontsize=14, pad=20)

        # Añadir leyenda
        import matplotlib.patches as mpatches
        humid_patch = mpatches.Patch(color='blue', label='Húmedo')
        dry_patch = mpatches.Patch(color='red', label='Seco')
        plt.legend(handles=[humid_patch, dry_patch],
                  loc='lower right',
                  bbox_to_anchor=(0.95, 0.05),
                  frameon=True,
                  framealpha=1,
                  edgecolor='black')

        plt.grid(True, alpha=0.3, axis='y')
        plt.xticks(fontstyle='italic', fontweight='bold', fontsize=8)
        plt.yticks(fontstyle='italic', fontweight='bold', fontsize=8)

        # Añadir texto del índice
        #plt.text(0.02, 0.98, 'ÍNDICE ESTANDARIZADO DE EVAPOTRANSPIRACIÓN',
                #transform=plt.gca().transAxes,
                #fontsize=10, verticalalignment='top')

        plt.tight_layout()
        plt.savefig(f'{output_path}/ESTACION_{codigo}.png', dpi=400, bbox_inches='tight')
        plt.close()

        print(f"Procesada estación: {codigo}")

    except Exception as e:
        print(f"Error en estación {codigo}: {str(e)}")

# Procesar todas las estaciones únicas
estaciones_unicas = estaciones_jalmes_FL['Codigo'].unique()

for codigo in estaciones_unicas:
    procesar_estacion(codigo)

# Concatenar todos los dataframes y guardar el archivo único
if all_stations_data:
    combined_df = pd.concat(all_stations_data, ignore_index=True)
    combined_df.to_csv(f'{output_data_path}/todos_datos_spei12.csv', index=False)
    print("Archivo combinado generado: todos_datos_spei12.csv")
else:
    print("No hay datos para combinar.")


print("Procesamiento completado!")

Mounted at /content/drive
Procesada estación: 14002
Procesada estación: 14004
Procesada estación: 14005
Procesada estación: 14006
Procesada estación: 14007
Procesada estación: 14008
Procesada estación: 14009
Procesada estación: 14010
Procesada estación: 14011
Procesada estación: 14013
Procesada estación: 14014
Procesada estación: 14015
Procesada estación: 14016
Procesada estación: 14017
Procesada estación: 14018
Procesada estación: 14019
Procesada estación: 14020
Procesada estación: 14021
Procesada estación: 14023
Procesada estación: 14024
Procesada estación: 14025
Procesada estación: 14026
Procesada estación: 14027
Procesada estación: 14028
Procesada estación: 14029
Procesada estación: 14030
Procesada estación: 14031
Procesada estación: 14032
Procesada estación: 14033
Procesada estación: 14034
Procesada estación: 14035
Procesada estación: 14036
Procesada estación: 14037
Procesada estación: 14038
Procesada estación: 14039
Procesada estación: 14040
Procesada estación: 14042
Procesada es